Regression for predicting the price of the house

# Steps we will follow:

## 1. Define the objective
- defining the target 
- defining a acceptable error (we will take into account MAE for the error) → **So define Maximum MAE**

---

## 2. Select data for the model
- cleaning of that data (en principio no va a hacer falta)
- Detectar outliers que puedan romper el modelo

---

## 3. EDA para detectar si un linear regression va a ser suficiente: justificarlo y aplicar un linear regression para demostrarlo

### 3.1 Univariado
- Distribución del target.
- Distribución y estadísticas de las features.

### 3.2 Bivariado
- Correlaciones numéricas (heatmap).
- ANOVA / boxplots para categóricas vs target.

### 3.3 Detectar no linealidades
- Scatterplots X vs Y.
- Transformaciones (log, sqrt) si una relación no es lineal.
- Crear polinómicas si es necesario.

> 👉 Aquí decides si la regresión lineal tiene sentido o necesitas algo más poderoso.

---

### ✔️ ¿Cómo elegir definitivamente?
Se hace así:

1️⃣ Divide los datos (train/test)  
2️⃣ Ejecuta cross-validation con GridSearch:  

- Lasso  
- Ridge  
- ElasticNet  

El que dé mejor MAE o RMSE → ese gana.  

➡️ No se decide a ojo. Se decide con CV.

---

## 4. Validar Supuestos: (aplicar lasso directamente y verificar)
- Linealidad
- Independencia de residuos

---

## 5. Preprocesing
- Dividir train y test
- Scaled, categorical-variables...

---

## 6. Entrenar y probar con varios modelos (usando cross validation)

### Entrenar y probar con:
- ✔️ RANSAC  
  Para outliers, recomendado

- ✔️ Theil-Sen  
  Robusto a ruido, muy usado en datos inestables

- ✔️ HuberRegressor  
  Mezcla LR con robustez

- ✔️ RandomForestRegressor  
  No lineal, buen baseline potente

- ✔️ CatBoost  
  De lo mejor en tabular  
  No requiere escalado  
  Maneja categóricas

- ✔️ XGBoost

- ✔️ LightGBM  

Casi siempre da un rendimiento brutal

---

### Evaluar métricas
- R², MAE
- Comprobar si hay overfitting o underfitting en cada modelo:  
    - Overfitting → test mucho peor que train  
    - Underfitting → ambos malos  
    - Buen modelo → train ≈ test y métricas buenas  

> 👉 El test manda; si test es bueno → el modelo generaliza.

---

## 7. Diagnóstico final (para verificar si el modelo que hemos elegido esta bien)
- Residuals vs predicted
- Predicted vs actual
- Q-Q plot
- Importancia de variables
- Coeficientes interpretables (si es lineal)














### IMPORTS

In [36]:
import pandas as pd
import numpy as np
import statistics as st

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack


### LOAD DATA

In [37]:
#load the data

data = pd.read_csv('../data/cleaned/data_final.csv')
dataCpy = data.copy()

### 1: Define the objective of the regression: 
- Target feature: price
- Now we will justify which will, what we will consider an acceptable maximum MAE

In [38]:
#Since the target feature is the price, the MAE will be in € units.

#AN OPTION WOULD BE: 

#1.We will get which is the standard deviation of "price" feature 
std_price = dataCpy["price"].std()
print(f"Standard deviation of the feature 'price': {round(std_price,2)}€") #--> The values of "price" differ from the mean: 310649.51€

#2.The maximum accepatble MAE could be a percentage of that standard deviation value. 
#In the house business, 31.000€ more or less expensive may have an impact on buyers, however a price of 10.000€ may not have so much impact. 
#So an acceptale MAE will be 10.000€ for all the predictions. 





Standard deviation of the feature 'price': 310649.51€


HOWEVER: 

The first option would be easier to implement (1 fixed maximum MAE for all the predictions). However, this would not be true. 
Since 10.000€ may not have an impact (would be an acecptable MAE) for a house of 500.000€ for example, but would have a big 
impact (would NOT be an acceptable MAE) for a house of 50.000€.

So we will take into account the "relative MAE" with an error of 5%. So for example: 

In [39]:

print(f"Cheapest House: {round(min(dataCpy["price"]),2)}€")
print(f"5% = {round(min(dataCpy["price"])*0.05,2)}€\n")

print(f"Most Expensive House: {round(max(dataCpy["price"]),2)}€")
print(f"5% = {round(max(dataCpy["price"])*0.05,2)}€\n")

print(f"Average price House: {round(st.mean(dataCpy["price"]),2)}€")
print(f"5% = {round(st.mean(dataCpy["price"])*0.05,2)}€\n")


Cheapest House: 50000.0€
5% = 2500.0€

Most Expensive House: 2500000.0€
5% = 125000.0€

Average price House: 477005.01€
5% = 23850.25€



### 2: Data cleaning for the model

In [40]:
#1.We will check if there are missing values (there should not be any, since we have previously done data cleaning)
print(f"Missing values:\n{dataCpy.isnull().sum()}") #0 missing values in target and the other features PERFECT




Missing values:
url                  0
price                0
zone                 0
neighborhood         0
built_area           0
usable_area          0
bedrooms             0
bathrooms            0
floor                0
exterior             0
elevator             0
garage               0
storage_room         0
balcony              0
new                  0
condition            0
year                 0
agency               0
consumption_label    0
emissions_label      0
description          8
dtype: int64


In [48]:
#2.We will get the features which the model will be working with (initally)

print(f"All features: {dataCpy.columns}\n\n")

#We will not take into account: Url and price (is the target) 

target_price = dataCpy.price
data_regresion = dataCpy.drop(["url","price"], axis=1)
print(f"Initially selected features: {data_regresion.columns}")


All features: Index(['url', 'price', 'zone', 'neighborhood', 'built_area', 'usable_area',
       'bedrooms', 'bathrooms', 'floor', 'exterior', 'elevator', 'garage',
       'storage_room', 'balcony', 'new', 'condition', 'year', 'agency',
       'consumption_label', 'emissions_label', 'description'],
      dtype='object')


Initially selected features: Index(['zone', 'neighborhood', 'built_area', 'usable_area', 'bedrooms',
       'bathrooms', 'floor', 'exterior', 'elevator', 'garage', 'storage_room',
       'balcony', 'new', 'condition', 'year', 'agency', 'consumption_label',
       'emissions_label', 'description'],
      dtype='object')


In [ ]:
#3.We will check if there are any outliers in the values of "price"
